- `ThreadPoolExecutor` (I/O-bound) and `ProcessPoolExecutor` (CPU-bound tasks)
- `Future` object represents the execution of the callable
- Error Handling - if the callable raises an exception, it can be caught when calling `result()` on the `Future` oject

https://realpython.com/python-gil/

Global Interpreter Lock (GIL)
- problem for **threads**
- not a problem for **processes** - each has their own GIL and memory space
- CPython implementation
- mutex - prevents deadlocks (as there is only one lock)
- only one thread holds the control of the Python interpreter
- allows for easier **integration with C libraries** (need for b-compatibility) that are not thread-safe, but this comes at the cost of limiting parallelism in multi-threaded applications
- effectively makes any CPU-bound Python program single-threaded
- problems source: reference counting `sys.getrefcount(a)`
- much impact on performance of **CPU-bound** programs, not much on I/O-bound
- acquire and release **overheads** added by the lock
- currently no better solutions that would not decrease the performace of single-threaded programs
- **starvation of I/O threads problem** 
    - in mixed I/O and CPU programs GIL can starve I/O-bound threads by creating a situation where they are unable to reacquire the GIL promptly due to the intensity of CPU-bound threads that reacquire for themselves immediately
    - added checking for chcking other threads
    - https://dabeaz.blogspot.com/2010/01/python-gil-visualized.html

Workarounds:
- multiprocessing - `multiprocessing` module creates separate processes that each have their own GIL and memory space
    - multiple processes are heavier than multiple threads, so, keep in mind that this could become a scaling bottleneck.
- Python implementations that do not have a GIL, such as Jython or IronPython (may lack compatibility)

Further reading: https://www.youtube.com/watch?v=Obt-vMVdM8s

In [1]:
import concurrent.futures
import time
import random

In [7]:
def do_work(seconds):
    print(f"Sleeping for {seconds} seconds")
    time.sleep(seconds)
    return f"Finished sleeping for {seconds} seconds"

In [8]:
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(do_work, random.randint(1, 5)) for _ in range(10)]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())

Sleeping for 5 seconds
Sleeping for 1 seconds
Sleeping for 3 seconds
Sleeping for 2 secondsFinished sleeping for 1 seconds

Sleeping for 1 secondsFinished sleeping for 3 seconds

Sleeping for 2 seconds
Finished sleeping for 2 seconds
Sleeping for 2 secondsFinished sleeping for 1 seconds

Sleeping for 1 secondsFinished sleeping for 5 seconds

Sleeping for 3 seconds
Finished sleeping for 2 seconds
Sleeping for 5 secondsFinished sleeping for 1 seconds

Finished sleeping for 2 seconds
Finished sleeping for 3 seconds
Finished sleeping for 5 seconds
